In [3]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')
import pymongo


In [4]:
f = "airbnb-listings.csv"
f_df = pd.read_csv(f, delimiter=';', encoding = 'latin-1')
f_df.head()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,2501888,https://www.airbnb.com/rooms/2501888,20170306202425,2017-03-07,Private bed in home,Sofa bed in the media room on the first floor....,Modernist home with a strong influence from MC...,Sofa bed in the media room on the first floor....,none,Upscale area of West Lake Hills on a bluff ove...,...,10.0,9.0,9.0,NaN,NaN,flexible,1,2.42,"30.2863434604, -97.8104775775","Host Has Profile Pic,Host Identity Verified,Is..."
1,377609,https://www.airbnb.com/rooms/377609,20170306202425,2017-03-07,Lake House minutes to Downtown,NaN,"Located just 7 minutes to Downtown, and 4 minu...","Located just 7 minutes to Downtown, and 4 minu...",none,NaN,...,10.0,10.0,9.0,NaN,NaN,strict,1,0.54,"30.3028051776, -97.7942255855","Host Has Profile Pic,Host Identity Verified,Is..."
2,15643010,https://www.airbnb.com/rooms/15643010,20170306202425,2017-03-07,Modern New Apartment Close to Everything!,"Close to the Barton Creek Greenbelt, S Lamar, ...",NaN,"Close to the Barton Creek Greenbelt, S Lamar, ...",none,NaN,...,10.0,10.0,10.0,NaN,NaN,moderate,1,1.86,"30.2521539737, -97.7995481364","Host Has Profile Pic,Host Identity Verified,Is..."
3,2236988,https://www.airbnb.com/rooms/2236988,20170306202425,2017-03-07,Austin Waterfront Retreat,Magnificent Austin waterfront home on Barton C...,Retreat within the city of Austin. 12 minutes...,Magnificent Austin waterfront home on Barton C...,none,Quiet neighborhood tucked away in Austin. Min...,...,10.0,10.0,10.0,NaN,NaN,strict,1,0.21,"30.2752712576, -97.843114474","Host Has Profile Pic,Is Location Exact"
4,14788276,https://www.airbnb.com/rooms/14788276,20170306202425,2017-03-07,Stay at the Domain's hotest residencial property.,Find indulgence the moment you enter the bouti...,NaN,Find indulgence the moment you enter the bouti...,none,NaN,...,10.0,10.0,9.0,NaN,NaN,moderate,1,0.69,"30.4001335924, -97.7241364202","Host Has Profile Pic,Is Location Exact"


In [5]:
f_df.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

In [6]:
n_df = f_df[[
    'Name','Street',
    'City', 'State', 'Zipcode','Property Type', 
    'Review Scores Rating']].copy()
n_df.head()

,Name,Street,City,State,Zipcode,Property Type,Review Scores Rating
0,Private bed in home,"Buckeye Trail, West Lake Hills, TX 78746, Unit...",West Lake Hills,TX,78746,Other,94.0
1,Lake House minutes to Downtown,"Westlake Dr, Austin, TX 78746, United States",Austin,TX,78746,House,96.0
2,Modern New Apartment Close to Everything!,"South MoPac Expressway, Austin, TX 78746, Unit...",Austin,TX,78746,Apartment,97.0
3,Austin Waterfront Retreat,"Bend O River Drive, Austin, TX 78746, United S...",Austin,TX,78746,House,96.0
4,Stay at the Domain's hotest residencial property.,"Esperanza Crossing, Austin, TX 78758, United S...",Austin,TX,78758,Condominium,93.0


In [7]:

n_df.rename(columns={'Zipcode':'Zip',
                     'Property Type':'Type',
                     'Number of Reviews':'Review Count',
                     'Street':'Address',
                     'Review Scores Rating':'Average Rating'}, 
                 inplace=True)

In [8]:
d = n_df.dropna()

In [9]:
d['State'] = d['State'].str.upper()

In [10]:
d.State.unique()


array(['TX', 'MA', 'CO', 'IL', 'NY', 'CA', 'LA', 'TN', 'OR', 'WA', 'MP',
       'DC', 'MD'], dtype=object)

In [11]:
d = d.set_index('Name')


In [12]:
d['Ratings2'] = d['Average Rating'] * (5/100)


In [13]:
d = d.drop(['Average Rating'],axis=1)
d.rename(columns={'Ratings2':'Average Rating'}, 
                 inplace=True)
d.head()

,Address,City,State,Zip,Type,Average Rating
Name,,,,,,
Private bed in home,"Buckeye Trail, West Lake Hills, TX 78746, Unit...",West Lake Hills,TX,78746,Other,4.70
Lake House minutes to Downtown,"Westlake Dr, Austin, TX 78746, United States",Austin,TX,78746,House,4.80
Modern New Apartment Close to Everything!,"South MoPac Expressway, Austin, TX 78746, Unit...",Austin,TX,78746,Apartment,4.85
Austin Waterfront Retreat,"Bend O River Drive, Austin, TX 78746, United S...",Austin,TX,78746,House,4.80
Stay at the Domain's hotest residencial property.,"Esperanza Crossing, Austin, TX 78758, United S...",Austin,TX,78758,Condominium,4.65


In [14]:
#open the connection to mongodb and call the database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.vacation 

In [15]:
for row in df.iterrows():
    db.accommodations.insert_one(
        {
            "Listing Name":row[0],
            "Address":row[1]['Address'],
            "City":row[1]['City'],
            "State":row[1]['State'],
            "Zip":row[1]['Zip'],
            "Average Rating":row[1]['Average Rating'],
            "Type":row[1]['Type']
        }
    )

NameError: name 'df' is not defined

In [17]:
print(db)


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacation')


In [18]:
myresults = list(db.accommodations.find())

In [22]:
myresults

[{'_id': ObjectId('5cb1f86ca36fa9263c98031e'),
  'Listing Name': 'Private bed in home',
  'Address': 'Buckeye Trail, West Lake Hills, TX 78746, United States',
  'City': 'West Lake Hills',
  'State': 'TX',
  'Zip': 78746.0,
  'Average Rating': 4.7,
  'Type': 'Other'},
 {'_id': ObjectId('5cb1f86ca36fa9263c98031f'),
  'Listing Name': 'Lake House minutes to Downtown',
  'Address': 'Westlake Dr, Austin, TX 78746, United States',
  'City': 'Austin',
  'State': 'TX',
  'Zip': 78746.0,
  'Average Rating': 4.800000000000001,
  'Type': 'House'},
 {'_id': ObjectId('5cb1f86ca36fa9263c980320'),
  'Listing Name': 'Modern New Apartment Close to Everything!',
  'Address': 'South MoPac Expressway, Austin, TX 78746, United States',
  'City': 'Austin',
  'State': 'TX',
  'Zip': 78746.0,
  'Average Rating': 4.8500000000000005,
  'Type': 'Apartment'},
 {'_id': ObjectId('5cb1f86ca36fa9263c980321'),
  'Listing Name': 'Austin Waterfront Retreat',
  'Address': 'Bend O River Drive, Austin, TX 78746, United Sta